<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# YahooFinance - Send daily prediction to Email
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/YahooFinance/YahooFinance_Send_daily_prediction_to_Email.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #yahoofinance #trading #markdown #prediction #plotly #slack #naas_drivers #notification #naas #investors #automation #analytics #email #html #image

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

With this template, you can get any ticker available in [Yahoo finance](https://finance.yahoo.com/quote/TSLA/), add predictions and send a daily email with naas.<br> 

## Input

### Get utils

In [1]:
# Load utils functions
from os import path

utils_path = "../utils/__utils__.ipynb"
if not path.exists(utils_path):
    utils_path = "utils/__utils__.ipynb"

%run "$utils_path"

✅ Notebook '../utils/naas_assets_helper.ipynb' loaded.
✅ Notebook '../utils/project_helper.ipynb' loaded.


### Import libraries

In [2]:
import json
import pandas as pd
import naas
import markdown2
from datetime import datetime
from IPython.core.display import display, HTML
from naas_drivers import naasauth

### Get params

In [3]:
ticker = PARAMS.get("TICKER")
data_point = PARAMS.get("DATA_POINT")
email_to = PARAMS.get("EMAIL_TO")

### Setup Email
👉 Here you can input your sender email and destination email.<br>
Note: emails are sent from notification@naas.ai by default

In [4]:
email_subject = f"📈 {ticker} predictions as of today"
email_from = naasauth.connect().user.me().get("username")

# Markdown template created on your local env
email_content_md = path.join(OUTPUTS_PATH, "email_content.md")

### Setup Inputs

In [5]:
csv_ticker = path.join(INPUTS_PATH, f"{ticker}.csv")
pipeline_path = path.join(MODELS_PATH, "__pipeline__.ipynb")
now = datetime.now().strftime("%Y-%m-%d")
excel_predict = path.join(OUTPUTS_PATH, f"{ticker}_{now}.xlsx")

### Get assets link

In [6]:
asset_name = "Plotly linechart"

data = get_naas_assets(asset_name)
link_html = data.get("HTML_LINK")
link_image = data.get("IMAGE_LINK")

## Model

### Get dataset from Yahoo Finance

In [7]:
df_yahoo = pd.read_csv(csv_ticker)

# Display dataframe
print("☑️ Rows fetched:", len(df_yahoo))
df_yahoo.tail(1)

☑️ Rows fetched: 70


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
69,2022-08-31,280.619995,281.200012,271.809998,276.149994,276.149994,27439995,TSLA


### Get prediction

In [8]:
df_predict = pd.read_excel(excel_predict)

# Display dataframe
print("☑️ Rows fetched:", len(df_predict))
df_predict.head(1)

☑️ Rows fetched: 91


,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,ARIMA,LINEAR,SVR,COMPOUND
0,0,2022-09-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.149994,282.196635,304.367525,287.571384


### Set actual data and variation

In [9]:
def get_variation(df):
    df = df.sort_values("Date", ascending=False).reset_index(drop=True)
    
    # Get value and value comp
    datanow = df.loc[0, "Close"]
    datayesterday = df.loc[1, "Close"]
    
    # Calc variation en value and %
    varv = datanow - datayesterday
    varp = (varv / datanow)
    
    # Format result
    datanow = "${:,.2f}".format(round(datanow, 1))
    datayesterday = "${:,.2f}".format(round(datayesterday, 1))
    varv = "{:+,.2f}".format(varv)
    varp = "{:+,.2%}".format(varp)
    return datanow, datayesterday, varv, varp

DATANOW, DATAYESTERDAY, VARV, VARP = get_variation(df_yahoo)
print("Value today:", DATANOW)
print("Value yesterday:", DATAYESTERDAY)
print("Var. in value:", VARV)
print("Var. in %:", VARP)

Value today: $276.10
Value yesterday: $277.70
Var. in value: -1.55
Var. in %: -0.56%


### Set predict data

In [10]:
def get_prediction(df, prediction):
    data = df.loc[0, prediction]
    
    # Format result
    data = "${:,.2f}".format(round(data, 1))
    return data

ARIMA = get_prediction(df_predict, "ARIMA")
print("Value ARIMA:", ARIMA)
SVR = get_prediction(df_predict, "SVR")
print("Value SVR:", SVR)
LINEAR = get_prediction(df_predict, "LINEAR")
print("Value LINEAR:", LINEAR)
COMPOUND = get_prediction(df_predict, "COMPOUND")
print("Value COMPOUND:", COMPOUND)

Value ARIMA: $276.10
Value SVR: $304.40
Value LINEAR: $282.20
Value COMPOUND: $287.60


### Create webhook to run your notebook again

In [11]:
link_webhook = naas.webhook.add(pipeline_path)

👌 Well done! Your Notebook has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Notebook as API" feature, just replace .add by .delete


### Create Email template 

In [12]:
%%writefile $email_content_md
Hello world,

The **TICKER** price is **DATANOW** right now, VARV vs yesterday (VARP).<br>
Yesterday close : DATAYESTERDAY

In +DATA_POINT days, basic ML models predict the following prices: 

- **arima**: ARIMA
- **svr**: SVR
- **linear**: LINEAR
- **compound**: COMPOUND
    
<img href=link_html target="_blank" src=link_image style="width:640px; height:360px;" /><br>
[Open dynamic chart](link_html)<br>

Please find attached the data in Excel.<br>

Have a nice day.
<br>

PS: You can [send the email again](link_webhook) if you need a fresh update.<br>
<div><strong>Full Name</strong></div>
<div>Open source lover | <a href="http://www.naas.ai/" target="_blank">Naas</a></div>
<div>+ 33 1 23 45 67 89</div>
<div><small>This is an automated email from my Naas account</small></div>

Overwriting /home/ftp/data-product-template/outputs/email_content.md


### Replace values in templates

In [13]:
def replace_value(md):
    post = md.replace("DATANOW", str(DATANOW))
    post = post.replace("TICKER", str(ticker))
    post = post.replace("DATAYESTERDAY", str(DATAYESTERDAY))
    post = post.replace("VARV", str(VARV))
    post = post.replace("VARP", str(VARP))
    post = post.replace("LINEAR", str(LINEAR))
    post = post.replace("SVR", str(SVR))
    post = post.replace("COMPOUND", str(COMPOUND))
    post = post.replace("ARIMA", str(ARIMA))
    post = post.replace("DATA_POINT", str(data_point))
    post = post.replace("link_image", str(link_image))
    post = post.replace("link_html", str(link_html))
    post = post.replace("link_webhook", str(link_webhook))
    return post

### Create Email content

In [14]:
content = open(email_content_md, "r").read()
md = markdown2.markdown(content)
email_content = replace_value(md)
display(HTML(email_content))

## Output

### Send Email

In [15]:
naas.notification.send(email_to=email_to,
                       subject=email_subject,
                       html=email_content,
                       files=[excel_predict],
                       email_from=email_from)

👌 💌 Email has been sent successfully !
